# Traditional machine learning-based approaches

## Naive Bayes

In [1]:
# Install library that can handle class imbalance
!pip install imbalanced-learn
!pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import random
import re
import string
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from imblearn.over_sampling import SMOTE

# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Function to set all seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

# Setting the seed
set_seed(42)

# Function to load a single dataset
def load_dataset(filename):
    df = pd.read_csv(filename)
    return df['text'], df['labels']

# Replace the numerical labels with the sentiment categories
def map_labels(label):
    if label == 0:
        return "negative"
    elif label == 1:
        return "neutral"
    elif label == 2:
        return "positive"
    else:
        return "unknown"

# List of Dutch stop words
dutch_stopwords = set(stopwords.words('dutch'))

# Preprocess text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if word not in dutch_stopwords]
    # Join tokens back into a single string
    text = ' '.join(tokens)
    return text

# Function to perform sentiment analysis and generate a classification report and confusion matrix
def cross_val_analysis(X_train_val, y_train_val, X_test, y_test):
    tfidf = TfidfVectorizer()
    X_train_val_tfidf = tfidf.fit_transform(X_train_val)
    X_test_tfidf = tfidf.transform(X_test)

    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_val_tfidf, y_train_val)

    nb_classifier = MultinomialNB()

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    y_pred_cv = cross_val_predict(nb_classifier, X_resampled, y_resampled, cv=skf)

    # Cross-validation classification report and confusion matrix
    report_cv = classification_report(y_resampled, y_pred_cv, zero_division=0)
    cm_cv = confusion_matrix(y_resampled, y_pred_cv, labels=["negative", "neutral", "positive"])

    # Train final model on the entire training+validation set and test on the unseen test set
    nb_classifier.fit(X_resampled, y_resampled)
    y_pred_test = nb_classifier.predict(X_test_tfidf)

    # Test set classification report and confusion matrix
    report_test = classification_report(y_test, y_pred_test, zero_division=0)
    cm_test = confusion_matrix(y_test, y_pred_test, labels=["negative", "neutral", "positive"])

    return report_cv, cm_cv, report_test, cm_test

# List of datasets paths
dataset_paths = ["1960s_gas.csv", "1970s_gas.csv", "1980s_gas.csv", "1990s_gas.csv"]

# Iterate over each dataset path in the list
for dataset_path in dataset_paths:
    dataset_name = dataset_path.split(".")[0]
    print(f"Processing {dataset_name}...")

    # Load dataset
    X, y = load_dataset(dataset_path)

    # Preprocess the text data
    X = X.apply(preprocess_text)

    # Map numerical labels to sentiment categories for ground truth
    y = y.apply(map_labels)

    # Split the dataset into 85% training+validation and 15% test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

    # Further split the training+validation set so that in the end there is a 70% training and 15% validation set
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)

    # Combine training and validation sets for cross-validation
    X_train_val_combined = pd.concat([X_train, X_val])
    y_train_val_combined = pd.concat([y_train, y_val])

    # Perform cross-validation analysis
    report_cv, cm_cv, report_test, cm_test = cross_val_analysis(X_train_val_combined, y_train_val_combined, X_test, y_test)

    # Print cross-validation classification report and confusion matrix
    print(f"Cross-Validation Classification Report for {dataset_name}:\n", report_cv)
    print(f"Cross-Validation Confusion Matrix for {dataset_name}:\n", cm_cv)
    print("-" * 50)

    # Print test set classification report and confusion matrix
    print(f"Test Set Classification Report for {dataset_name}:\n", report_test)
    print(f"Test Set Confusion Matrix for {dataset_name}:\n", cm_test)
    print("=" * 50)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing 1960s_gas...
Cross-Validation Classification Report for 1960s_gas:
               precision    recall  f1-score   support

    negative       0.79      0.93      0.86       187
     neutral       0.78      0.73      0.75       187
    positive       0.81      0.73      0.77       187

    accuracy                           0.80       561
   macro avg       0.80      0.80      0.79       561
weighted avg       0.80      0.80      0.79       561

Cross-Validation Confusion Matrix for 1960s_gas:
 [[174   9   4]
 [ 23 136  28]
 [ 22  29 136]]
--------------------------------------------------
Test Set Classification Report for 1960s_gas:
               precision    recall  f1-score   support

    negative       0.32      0.50      0.39        12
     neutral       0.33      0.25      0.29        20
    positive       0.58      0.55      0.56        33

    accuracy                           0.45        65
   macro avg       0.41      0.43      0.41        65
weighted avg       0

## Support Vector Machine

In [2]:
# Install library that can handle class imbalance
!pip install scikit-learn imbalanced-learn
!pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import random
import re
import string
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from imblearn.over_sampling import SMOTE
import nltk

# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Function to set all seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

# Setting the seed
set_seed(42)

# Function to load a single dataset
def load_dataset(filename):
    df = pd.read_csv(filename)
    return df['text'], df['labels']

# Replace the numerical labels with the sentiment categories
def map_labels(label):
    if label == 0:
        return "negative"
    elif label == 1:
        return "neutral"
    elif label == 2:
        return "positive"
    else:
        return "unknown"

# List of Dutch stop words
dutch_stopwords = set(stopwords.words('dutch'))

# Preprocess text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if word not in dutch_stopwords]
    # Join tokens back into a single string
    text = ' '.join(tokens)
    return text

# Function to perform sentiment analysis and generate classification report and confusion matrix
def cross_val_analysis(X_train_val, y_train_val, X_test, y_test):
    tfidf = TfidfVectorizer()
    X_train_val_tfidf = tfidf.fit_transform(X_train_val)
    X_test_tfidf = tfidf.transform(X_test)

    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_val_tfidf, y_train_val)

    svm_classifier = SVC(kernel='linear', random_state=42)  # Using linear kernel as an example, you can change it

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    y_pred_cv = cross_val_predict(svm_classifier, X_resampled, y_resampled, cv=skf)

    # Cross-validation classification report and confusion matrix
    report_cv = classification_report(y_resampled, y_pred_cv, zero_division=0)
    cm_cv = confusion_matrix(y_resampled, y_pred_cv, labels=["negative", "neutral", "positive"])

    # Train final model on the entire training+validation set and test on the unseen test set
    svm_classifier.fit(X_resampled, y_resampled)
    y_pred_test = svm_classifier.predict(X_test_tfidf)

    # Test set classification report and confusion matrix
    report_test = classification_report(y_test, y_pred_test, zero_division=0)
    cm_test = confusion_matrix(y_test, y_pred_test, labels=["negative", "neutral", "positive"])

    return report_cv, cm_cv, report_test, cm_test

# List of datasets paths
dataset_paths = ["1960s_gas.csv", "1970s_gas.csv", "1980s_gas.csv", "1990s_gas.csv"]

# Iterate over each dataset path in the list
for dataset_path in dataset_paths:
    dataset_name = dataset_path.split(".")[0]
    print(f"Processing {dataset_name}...")

    # Load dataset
    X, y = load_dataset(dataset_path)

    # Preprocess the text data
    X = X.apply(preprocess_text)

    # Map numerical labels to sentiment categories for ground truth
    y = y.apply(map_labels)

    # Split the dataset into 85% training+validation and 15% test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

    # Further split the training+validation set so that in the end there is 70% training and 15% validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)

    # Combine training and validation sets for cross-validation
    X_train_val_combined = pd.concat([X_train, X_val])
    y_train_val_combined = pd.concat([y_train, y_val])

    # Perform cross-validation analysis
    report_cv, cm_cv, report_test, cm_test = cross_val_analysis(X_train_val_combined, y_train_val_combined, X_test, y_test)

    # Print cross-validation classification report and confusion matrix
    print(f"Cross-Validation Classification Report for {dataset_name}:\n", report_cv)
    print(f"Cross-Validation Confusion Matrix for {dataset_name}:\n", cm_cv)
    print("-" * 50)

    # Print test set classification report and confusion matrix
    print(f"Test Set Classification Report for {dataset_name}:\n", report_test)
    print(f"Test Set Confusion Matrix for {dataset_name}:\n", cm_test)
    print("=" * 50)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing 1960s_gas...
Cross-Validation Classification Report for 1960s_gas:
               precision    recall  f1-score   support

    negative       0.91      0.92      0.92       187
     neutral       0.81      0.78      0.79       187
    positive       0.76      0.78      0.77       187

    accuracy                           0.83       561
   macro avg       0.83      0.83      0.83       561
weighted avg       0.83      0.83      0.83       561

Cross-Validation Confusion Matrix for 1960s_gas:
 [[172   5  10]
 [  5 145  37]
 [ 11  30 146]]
--------------------------------------------------
Test Set Classification Report for 1960s_gas:
               precision    recall  f1-score   support

    negative       0.30      0.25      0.27        12
     neutral       0.31      0.20      0.24        20
    positive       0.55      0.70      0.61        33

    accuracy                           0.46        65
   macro avg       0.39      0.38      0.38        65
weighted avg       0